# Predicting Topic of the Text

## Table of Content
<ol>
<li><a href="#intro">Introduction</a></li>
<li><a href="#lib">Importing Libraries</a></li>
<li><a href="#read">Read and Preprocess the Data</a></li>
<li><a href="#build">Building Sequential Model</a></li>
</ol>

<a id='intro'></a>
## Introduction
This project aims to train a Sequential Model to predict the topic of a given text. Using TensorFlow and Keras, we will preprocess text data, convert it into a suitable format, and train a neural network for accurate text classification.

### Dataset Overview
* The data set is split into two csv files:
    * **Train set:** 2,801 rows, 5 variables.
    * **Test set:** 701 rows, 5 variables.


### Data Variables:
* **ID:** A unique identifier for each text.
* **ID:** Another identifier for each text.
* **Text:** The text content for which we predict the topic.
* **Label:** The encoded topic of the text.
* **Target:** The actual topic name of the text.

<a id='lib'></a>
## Improting Libraries

In [85]:
import pandas as pd
import numpy as np
import re
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from nltk.corpus import words, stopwords
from nltk.tokenize import word_tokenize
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential

<a id='read'></a>
## Read and Preprocess the Data

In [86]:
# Reading train and test files each from its csv file
df_train = pd.read_csv("Data/train.csv")
df_test = pd.read_csv("Data/test.csv")

In [87]:
df_train

,Unnamed: 0,ID,text,labels,target
0,98,118,MENTION لن يقضي التطعيم على كورونا لأنه تطعيم...,0,Covid Vaccine
1,1425,1677,انشاء وزارة او هيئة التحول الالكتروني أصبحت ضر...,1,Digital Transformation
2,2938,3443,زمن تمكين المرأة 👍🏻,2,Women empowerment
3,3134,3677,ويسرنا دعوتكم لحضور ملتقى #سعوديات_تمكين_ورياد...,2,Women empowerment
4,371,439,البدء في تأهيل منسوبي جمعية العوامية الخيرية ل...,1,Digital Transformation
...,...,...,...,...,...
2796,2119,2489,حسبي الله على من مكنهم .. بأسم تمكين المراة...,2,Women empowerment
2797,610,730,اختي العزيزة قريت واعرف كل تفاصيل مسؤولة تم...,2,Women empowerment
2798,294,354,كورونا علم المؤسسات والشركات والحكومات أن التح...,1,Digital Transformation
2799,1188,1401,يعني الحين كورونا بينتهي اذا تطعم الشعب كله!!!...,0,Covid Vaccine


In [88]:
df_test

,Unnamed: 0,ID,text,labels,target
0,124,153,متوفر في عدد من المراكز الصحية ويمكن أخذه مباش...,0,Covid Vaccine
1,1149,1355,اتمنى منكم ان تستفيدوا من تطورهم التقني لان ا...,1,Digital Transformation
2,1365,1612,تمكين المرأه,2,Women empowerment
3,2592,3036,ترك الإعلام للبرامج الترفيهية والمقابلات التاف...,1,Digital Transformation
4,544,647,في اليوم الدولي للمرأة والفتاة في ميدان العلوم...,2,Women empowerment
...,...,...,...,...,...
696,2894,3398,مشاركة المراة في عملية التنمية ، أصبحت حتمية ...,2,Women empowerment
697,1163,1371,#ساعد_زوجتك_بالمطبخ فالقانون الجديد يعتني بحق...,2,Women empowerment
698,2009,2362,نحن في عصر تمكين المرأة في زمن سيدنا محمد بن...,2,Women empowerment
699,764,910,التحول التقني الي تعيشة المملكة هالفتره غير م...,1,Digital Transformation


Next, we will define the necessary functions to clean the text, making it suitable for training the model.

In [89]:
import nltk
nltk.download('words')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [90]:

# Defining a set that contains arabic stop words to use it in cleaning the text
arabic_stopwords = set([
    "في", "من", "على", "و", "عن", "أن", "إلى", "ب", "مع", "هذا", "كما",
    "ذلك", "لكن", "كل", "أي", "أنا", "هو", "هي", "هم", "هن", "التي", "الذي",
    "الذين", "عند", "كان", "ليس", "لم", "بعد", "أو", "إذا", "بين", "حتى",
    "ثم", "له", "لها", "ما", "متى", "هناك", "هنا", "هي", "هو", "كلها", "كلهم"
])

# Clean text to keep words
def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)
    return text

# Lower Case English letters
def to_lower(text):
    return ' '.join(word.lower() if word.isascii() else word for word in text.split())

def remove_stopwords(text):
    stop_words = set(stopwords.words('english')) | arabic_stopwords
    words2 = word_tokenize(text)
    return ' '.join([word for word in words2 if word not in stop_words])

def preprocess_text(text):
    text = clean_text(text)
    text = to_lower(text)
    preprocessed_text = remove_stopwords(text)
    return text

Now that the cleaning functions are ready, we will apply them to the text as a preprocessing step. Since the goal is to predict the topic of the text, the only independent variable is the text itself.

In [91]:
# Applying cleaning function on the independent variable
from nltk.corpus import words
df_train['text'] = df_train['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

In [92]:
df_train

,Unnamed: 0,ID,text,labels,target
0,98,118,mention لن يقضي التطعيم على كورونا لأنه تطعيم ...,0,Covid Vaccine
1,1425,1677,انشاء وزارة او هيئة التحول الالكتروني أصبحت ضر...,1,Digital Transformation
2,2938,3443,زمن تمكين المرأة,2,Women empowerment
3,3134,3677,ويسرنا دعوتكم لحضور ملتقى سعوديات_تمكين_ورياده...,2,Women empowerment
4,371,439,البدء في تأهيل منسوبي جمعية العوامية الخيرية ل...,1,Digital Transformation
...,...,...,...,...,...
2796,2119,2489,حسبي الله على من مكنهم بأسم تمكين المراة,2,Women empowerment
2797,610,730,اختي العزيزة قريت واعرف كل تفاصيل مسؤولة تمكين...,2,Women empowerment
2798,294,354,كورونا علم المؤسسات والشركات والحكومات أن التح...,1,Digital Transformation
2799,1188,1401,يعني الحين كورونا بينتهي اذا تطعم الشعب كله ول...,0,Covid Vaccine


In [93]:
df_test

,Unnamed: 0,ID,text,labels,target
0,124,153,متوفر في عدد من المراكز الصحية ويمكن أخذه مباش...,0,Covid Vaccine
1,1149,1355,اتمنى منكم ان تستفيدوا من تطورهم التقني لان ال...,1,Digital Transformation
2,1365,1612,تمكين المرأه,2,Women empowerment
3,2592,3036,ترك الإعلام للبرامج الترفيهية والمقابلات التاف...,1,Digital Transformation
4,544,647,في اليوم الدولي للمرأة والفتاة في ميدان العلوم...,2,Women empowerment
...,...,...,...,...,...
696,2894,3398,مشاركة المراة في عملية التنمية أصبحت حتمية تمل...,2,Women empowerment
697,1163,1371,ساعد_زوجتك_بالمطبخ فالقانون الجديد يعتني بحقوق...,2,Women empowerment
698,2009,2362,نحن في عصر تمكين المرأة في زمن سيدنا محمد بن س...,2,Women empowerment
699,764,910,التحول التقني الي تعيشة المملكة هالفتره غير مس...,1,Digital Transformation


In [94]:
# Defining the series of independent variable of the training and testing set
X_train = df_train.text
X_test = df_test.text

### Tokenizing
Before training, we tokenize the text using TensorFlow's `Tokenizer`.

In [95]:
num_words = 1000
tokenizer = Tokenizer(num_words=num_words, lower=False)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_matrix(X_train, mode="tfidf")
X_test = tokenizer.texts_to_matrix(X_test, mode="tfidf")

Now that the independent variables are ready, we'll define the dependent variable (labels). Since this is a multi-class classification problem, we’ll one-hot encode the labels using `to_categorical`

In [96]:
num_classes = 3
y_train = to_categorical(df_train.labels, num_classes=num_classes)
y_test = to_categorical(df_test.labels, num_classes=num_classes)

<a id='build'></a>
## Building Sequential Model

In [97]:
# Building the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(num_words,)),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(3),
    tf.keras.layers.Activation("sigmoid")
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [98]:
# Compiling model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [99]:
# Training model
batch_size = 128
epochs = 10
history = model.fit(X_train, y_train, epochs=epochs, verbose=1)

Epoch 1/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.7943 - loss: 0.5131
Epoch 2/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9990 - loss: 0.0114
Epoch 3/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.9987 - loss: 0.0056
Epoch 4/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9996 - loss: 0.0028
Epoch 5/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9999 - loss: 0.0017
Epoch 6/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9998 - loss: 0.0011
Epoch 7/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9995 - loss: 0.0013
Epoch 8/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 1.0000 - loss: 7.4229e-04
Epoch 9/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 1.0000 - loss: 5.0444e-04
Epoch 10/10
88/88 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 1.0000 - loss: 6.6148e-04


In [100]:
# Evaluating model
score = model.evaluate(X_test, y_test, verbose = 0)
print(score[0])
print(score[1])

0.0037500059697777033
1.0


In [101]:
print('Test set\nLoss: {:0.15f}\nAccuracy: {:0.15f}'.format(score[0],score[1]))

Test set
Loss: 0.003750005969778
Accuracy: 1.000000000000000
